In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import undetected_chromedriver as uc
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
import pandas as pd
import os
import time

from crypto_web_scraping.scrap import *

In [4]:
driver = create_driver()
df = get_basic_info(driver)

In [ ]:
df

In [ ]:
from selenium import webdriver
url = “https://finance.yahoo.com/topic/stock-market-news"
links =[]
for i in range(1,10):
    #define a driver
    driver = webdriver.Chrome(‘C:\chromedriver’) #use the path in your computer
    driver.implicitly_wait(5)
    #get request
    driver.get(url)
    #keep clicking each news article and grab their url
    elem = driver.find_element_by_css_selector(“#Fin-Stream > ul > li:nth-child({})”.format(i))
    #get the URL after click
    elem.click()
    #store in a list
    links.append(driver.current_url)

In [11]:
url = "https://finance.yahoo.com/topic/stock-market-news"
driver = create_driver(url)
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="consent-page"]/div/div/div/form/div[2]/div[2]/button[1]'))).click()
driver.title

'Latest Stock Market News'

In [52]:
url = 'https://finance.yahoo.com/quote/TSLA?p=TSLA&.tsrc=fin-srch'
driver = create_driver(url)
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="consent-page"]/div/div/div/form/div[2]/div[2]/button[1]'))).click()

driver.implicitly_wait(15)

driver.execute_script("window.scrollTo(0, 1000000000000);")

In [53]:
#driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
articles = driver.find_elements(By.CLASS_NAME, 'js-stream-content.Pos\(r\)')
len(articles)

10

In [24]:
articles = driver.find_elements(By.CLASS_NAME, 'Ov\(h\).Pend\(44px\).Pstart\(25px\)')
len(articles)

9

In [56]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
import time
import pandas as pd

url = "https://finance.yahoo.com/topic/stock-market-news"
links = [] 
headers = {'accept-ranges':'bytes',
          'access-control-allow-origin':'*',
          'content-encoding': "gzip",
          'cache-control': 'private, max-age=86400',
          'content-type':'text/javascript'}
info = []


def get_links(url):
    driver = create_driver(url)
    driver.find_element(By.XPATH, '//*[@id="consent-page"]/div/div/div/form/div[2]/div[2]/button[1]').click()
    element = driver.find_element(By.XPATH, "//div[@id='Col2-6-LinkOut-Proxy']")

    actions = ActionChains(driver)
    actions.move_to_element(element).perform()
    time.sleep(3)
        
    all_a = driver.find_elements(By.XPATH, '//h3[@class="Mb(5px)"]//a')

    for a in all_a: 
        if a.get_attribute('href') not in links:
            links.append(a.get_attribute('href'))
                
                
    driver.close()
    print("Number of links that are gathered : {}".format(len(links)))
    
    return links



def get_info(url):
    #grab the information
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'lxml')

    news = soup.find('div', attrs={'class': 'caas-body'}).text
    headline = soup.find('h1').text 
    date = datetime.strptime(soup.find('time').text, "%B %d, %Y, %I:%M %p")
    
    #combine all info into a list of columns
    columns = [news, headline, date, url]
    #give columns names
    column_names = ['News','Headline','Date', 'Url']
    
    return dict(zip(column_names, columns))

In [57]:

links =  get_links(url)


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@id='Col2-6-LinkOut-Proxy']"}
  (Session info: headless chrome=103.0.5060.53)
Stacktrace:
#0 0x5568170feb13 <unknown>
#1 0x556816f05688 <unknown>
#2 0x556816f3ccc7 <unknown>
#3 0x556816f3ce91 <unknown>
#4 0x556816f6fe34 <unknown>
#5 0x556816f5a8dd <unknown>
#6 0x556816f6db94 <unknown>
#7 0x556816f5a7a3 <unknown>
#8 0x556816f300ea <unknown>
#9 0x556816f31225 <unknown>
#10 0x5568171462dd <unknown>
#11 0x55681714a2c7 <unknown>
#12 0x55681713022e <unknown>
#13 0x55681714b0a8 <unknown>
#14 0x556817124bc0 <unknown>
#15 0x5568171676c8 <unknown>
#16 0x556817167848 <unknown>
#17 0x556817181c0d <unknown>
#18 0x7f7d3941e609 <unknown>


In [ ]:
for link in links:
    try:
        scraped = get_info(link)
        info.append(scraped)
        time.sleep(3)
        
    except requests.exceptions.ConnectionError:
        print("Connection refused by the server..")
        print("Let me sleep for 3 seconds")
        print("ZZzzzz...")
        time.sleep(3)
        print("Was a nice sleep, now let me continue...")

df = pd.DataFrame(info)
df.to_csv("finance_yahoo_news.csv", index=False)